In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
using SparseArrays
using LinearAlgebra
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/vectorize.jl")

In [ ]:
#load borders
border = import_border("../data/borders/border.json");

In [ ]:
# create the lattice grid
dx = 20.0
Nx, Ny, xrange, yrange, isinside, isborder, n, isgrid = get_grid(border, Float64(dx));

In [ ]:
plot(heatmap(xrange, yrange, 2 * isinside + isborder))

In [ ]:
heatmap(xrange,yrange,isgrid-isborder-isinside)

In [ ]:
# compute or load grid parameters
recompute_param = true
sigma = 150.0
if(recompute_param || !isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params(isinside, isborder, n, Float64(dx), yrange, xrange, "../data/disc/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", sigma=sigma)
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5");
end

In [ ]:
plot(hm_plot(isgrid, xrange, yrange, bx), hm_plot(isgrid, xrange, yrange, by),
    hm_plot(isgrid, xrange, yrange, m), hm_plot(isgrid, xrange, yrange, d), hm_plot(isgrid, xrange, yrange, p),
    layout=(5,1), size=(700,1500))

In [ ]:
# compute or load grid parameters
recompute_param = true
# sigma = 100.0
if(recompute_param || !isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params_diff(isinside, n, Float64(dx), yrange, xrange, "../data/disc/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", Niter = 10000, dmax = 10.0, patch=0.003, bmin=0.01)
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5");
end

In [ ]:
th0, ~ = compute_stable_sol(isinside, n, bx, by, p);

In [ ]:
hm_plot(isgrid, xrange, yrange, th0)

In [ ]:
m = 1e-5 * ones(Ny, Nx)
d = 0.6 * m
bx = 8 * ones(Ny, Nx)
by = 8 * ones(Ny, Nx)
p = zeros(Ny, Nx)
m[.!isgrid] .= 0
d[.!isgrid] .= 0
p[.!isgrid] .= 0

In [ ]:
isinsideflat, bxflat, byflat, pflat, minvflat, gammaflat, bflat, xneigh, yneigh = vectorize(isinside, n, bx, by, p, m, d);

In [ ]:
minimum(minvflat[isinsideflat])

In [ ]:
# define a disturbance
dP = -9.0
# dP = 0.0
sigma = 200.0
location = [-1500.0, -900.0]
dp = local_disturbance(isgrid, xrange, yrange, location, dP, sigma)
dpflat = vec(dp)
println("Synchronized frequency: ", trapz((yrange, xrange), p .+ dp) / trapz((yrange, xrange), d))
hm_plot(isinside, xrange, yrange, dp)


In [ ]:
A = zeros(Nx * Ny, Nx * Ny)
#Threads.@threads for i in 1:Nx*Ny
mflat = vec(m)
for i in 1:Nx*Ny
    if(isinsideflat[i])
        A[i, i-Ny] = bxflat[i-Ny] / mflat[i]
        A[i, i+Ny] = bxflat[i] / mflat[i]
        A[i, i-1] = byflat[i-1] / mflat[i]
        A[i, i+1] = byflat[i] / mflat[i]
        A[i, i] = -(bxflat[i-Ny] + bxflat[i] + byflat[i-1] + byflat[i]) / mflat[i]
    end
end

#Threads.@threads for k in 1:size(n, 1)
for k in 1:size(n, 1)
    # with Nx goes out of bound
    i = (Int64(n[k, 2]) - 1) * Ny + Int64(n[k, 1])
    nx = n[k, 4] 
    ny = n[k, 3]
    A[i, i-Ny] = (1 + nx) * bxflat[i-Ny] / mflat[i]
    A[i, i+Ny] = (1 - nx) * bxflat[i] / mflat[i]
    A[i, i-1] = (1 + ny) * byflat[i-1] / mflat[i]
    A[i, i+1] = (1 - ny) * byflat[i] / mflat[i]
    A[i, i] = -((1 + ny) * byflat[i-1] + (1 - ny) * byflat[i] + (1 - nx) * bxflat[i] + (1 + nx) * bxflat[i-Ny]) / mflat[i]
end  
A = sparse(A)

In [ ]:
isflat = vec(isgrid);

In [ ]:
# perform a dynamical simulation
mflat = vec(m)[isflat]
dflat = vec(d)[isflat]
interval = 1
dt = 0.05
Ndt = 1000

println("Total time: ", dt * Ndt)

Nn = sum(isflat)

omegas = zeros(Nx * Ny,1 + Int64(ceil(Ndt/interval)))
ts = zeros(1 + Int64(ceil(Ndt/interval)))
th = [copy(vec(th0)[isflat]); zeros(Nn)]
# th = zeros(2 * Nn)
C = [Matrix(1.0I, Nn, Nn) Matrix(dt/2 * I, Nn, Nn);
    dt / 2 * A[isflat, isflat] / dx^2 (-dt/2 * Diagonal(dflat./mflat) + Matrix(1.0I, Nn, Nn))]
D = [Matrix(1.0I, Nn, Nn) Matrix(-dt/2 * I, Nn, Nn);
    -dt/2 * A[isflat, isflat] / dx^2 (dt/2 * Diagonal(dflat./mflat) + Matrix(1.0I, Nn, Nn))]
P = [zeros(Nn); dt * (pflat[isflat] .+ dpflat[isflat]) ./ mflat]
# println(size(th))
# println(size(P))

@time begin
    for t in 1:Ndt
        v = C * th + P
        th = D\v
        if(mod(t,interval) == 0)
            println("NIter: ", t)
            omegas[isflat,Int64(t/interval) + 1] = th[Nn+1:end]
            ts[Int64(t/interval) + 1] = t*dt
        end
    end
end
# Rewrite omegas and thetas in 2d format
omegasre = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
for i=1:1 + Int64(ceil(Ndt/interval))
    for j=1:Nx*Ny
        omegasre[(j-1) % Ny + 1, (j-1) ÷ Ny + 1, i] = omegas[j, i]
    end
end

In [ ]:
start=1
plot(ts,omegasre[20,20,start:end])
plot!(ts,omegasre[60,80,start:end])
plot!(ts,omegasre[10,100,start:end])
plot!(ts,omegasre[24,121,start:end])
xlabel!("\$t[s]\$")
ylabel!("\$\\omega[1/s]\$")

In [ ]:
@gif for i=1:size(omegasre,3)
# @gif for i=1:30
#     do_plot(isinside, omegas[:,:,i])
    hm_plot(isinside, xrange, yrange, omegasre[:,:,i], clim=(minimum(omegasre), maximum(omegasre)))
    #contour(omegas[:,:,i],fill=true,levels= -2:0.1:2)
    #contour(omegas[:,:,i],fill=true)
end

In [ ]:
mean(omegasre[isgrid, 1 + Int64(ceil(Ndt/interval))])

In [ ]:
sum(dpflat)